In [1]:
import pandas as pd
import numpy as np
from pulp import *

In [2]:
year = '2010'
df = pd.read_excel(open('RawData_Price.xlsx', 'rb'), sheet_name=year)
X_df = df[['Coal']]
X = X_df.to_numpy(dtype=np.float64)
Y_df = df[['Electricity']]
Y = Y_df.to_numpy(dtype=np.float64)
B_df = df[['CO2 Emission', 'SO2 Emission', 'NOX Emission']]
B = B_df.to_numpy(dtype=np.float64)
P_df = df[['Price']]
P = P_df.to_numpy(dtype=np.float64)

In [3]:
I = [i for i in range(1)]
J = [j for j in range(1)]
Q = [q for q in range(3)]
K = [k for k in range(48)]
R = [r for r in range(48)]

Q_star = [0, 1, 2] 
Q_no_star = [] 

## Step 1: DEA with direction

In [4]:
def DDEA(Xr, Yr, Br, gY, gB):
    Eff = LpVariable('eff', lowBound=None, upBound=None, cat='Continuous')
    _lambda = LpVariable.dicts('_lambda', (K), lowBound=0, upBound=None, cat='Continuous')
    _mu = LpVariable.dicts('_mu', (K), lowBound=0, upBound=None, cat='Continuous')

    model1 = LpProblem('model1', LpMaximize)
    
    # Objective function
    model1 += Eff

    # Constraints
    # I Constraint
    model1 += lpSum((_lambda[k]+_mu[k])*X[k] for k in K) <= Xr
    # GO Constraint
    model1 += lpSum(_lambda[k]*Y[k] for k in K) >= Yr + Eff*gY
    # BO Constraint
    for q in Q_star:
        model1 += lpSum(_lambda[k]*B[k,q] for k in K) <= Br[q] - Eff*gB[q]
    # Convex
    model1 += lpSum(_lambda[k]+_mu[k] for k in K) == 1

    # solve 
#     solver = getSolver('GUROBI')
    model1.solve()
    return (LpStatus[model1.status], value(model1.objective))

In [5]:
a = 0.5
g_Coal = 0
g_Elec = 1*a
g_CO2, g_SO2, g_NOX = .048*(1-a), .508*(1-a), .444*(1-a)
g_B = [g_CO2, g_SO2, g_NOX]
result = []
for r in R:
    Xr, Yr = X[r,0], Y[r,0]
    Br = B[r]
    result.append(DDEA(Xr, Yr, Br, g_Elec, g_B))

_Eff_ = [result[k][1] for k in K]
_Eff_

[7767.9383,
 60569.461,
 70811.596,
 0.0,
 0.0,
 56300.116,
 0.0,
 9999.7009,
 23502.641,
 49749.129,
 762.0201,
 96084.735,
 0.0,
 56997.825,
 35450.439,
 40598.513,
 72341.29,
 24129.791,
 0.0,
 13527.484,
 0.0,
 138201.75,
 71330.499,
 30826.461,
 35538.306,
 9516.5743,
 0.0,
 153155.97,
 108659.62,
 6761.2675,
 8237.611,
 0.0,
 8797.0948,
 32750.38,
 0.0,
 138523.14,
 25097.822,
 0.0,
 0.0,
 41409.11,
 14624.839,
 0.0,
 0.0,
 81674.501,
 0.0,
 57444.86,
 0.0,
 86438.445]

## Step 2: Poject to frontier $Y+\eta g^Y$、$B-\eta g^B$

In [6]:
g_B = np.array(g_B)
_ftY_ = np.array([Y[k]+_Eff_[k]*g_Elec for k in K])
_ftB_ = np.array([B[k]-(_Eff_[k]*g_B) for k in K])

In [7]:
# Normalize
NX = X[:,0] / max(X[:,0])
NY = Y[:,0] / max(Y[:,0])
NB = (np.array([B[:,0]/max(B[:,0]), B[:,1]/max(B[:,1]), B[:,2]/max(B[:,2])])).T

_NftY_ = _ftY_[:,0] / max(_ftY_[:,0])
_NftB_ = (np.array([_ftB_[:,0]/max(_ftB_[:,0]), _ftB_[:,1]/max(_ftB_[:,1]), _ftB_[:,2]/max(_ftB_[:,2])])).T

## Step 3: DMP

In [8]:
def DDEA_Dual(NX, NY, NB, NXr, NftYr, NftBr, gY, gB):
    v = LpVariable('v', lowBound=0, upBound=None, cat='Continuous')
    u = LpVariable('u', lowBound=0, upBound=None, cat='Continuous')
    u0 = LpVariable('u0', lowBound=None, upBound=None, cat='Continuous')
    w = LpVariable.dicts('w', (Q), lowBound=None, upBound=None, cat='Continuous')

    model2 = LpProblem('model2', LpMinimize)
    
    # Objective function
    model2 += v

    # Constraints
    model2 += v*NXr - u*NftYr + lpSum(w[q]*NftBr[q] for q in Q_star) + u0 == 0
    for k in K:
        nb = NB[k]
        model2 += v*NX[k] - u*NY[k] + lpSum(w[q]*nb[q] for q in Q_star) + u0 >= 0
        model2 += v*NX[k] +u0 >= 0
    model2 += u*gY + lpSum(w[q]*gB[q] for q in Q_star) == 1

    # solve 
#     solver = getSolver('GUROBI')
    model2.solve()
    return (LpStatus[model2.status], value(model2.objective))

In [9]:
# GMP
result2 = []
for r in R:
    NXr = NX[r]
    NftYr = _NftY_[r]
    NftBr = _NftB_[r]
    result2.append(DDEA_Dual(NX, NY, NB, NXr, NftYr, NftBr, 1, [0,0,0]))

_GMPv = [result2[k][1] for k in K]
_GMPv

[0.0,
 0.0,
 0.0,
 0.0,
 0.99390919,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.089585,
 0.0,
 0.0,
 0.0,
 0.53404268,
 0.0,
 0.19426866,
 0.0,
 0.0,
 0.0,
 0.96916407,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.26800151,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.38943173,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [10]:
# BMP
result3 = []
for r in R:
    NXr = NX[r]
    NftYr = _NftY_[r]
    NftBr = _NftB_[r]
    result3.append(DDEA_Dual(NX, NY, NB, NXr, NftYr, NftBr, 0, [.048,.508,.444]))

_BMPv = [result3[k][1] for k in K]
_BMPv

[0.0,
 0.0,
 0.0,
 0.0,
 0.10367704,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 4.1955885,
 0.0,
 0.0,
 0.0,
 4.8743917,
 0.0,
 0.98592516,
 0.0,
 0.0,
 0.0,
 2.2966938,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 8.5150393,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.41558733,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [11]:
# MP
g = np.array([1*max(Y[:,0]), .048*max(B[:,0]), .508*max(B[:,1]), .444*max(B[:,2])])
_DMP_ = []
for r in R:
    nv = _BMPv[r]/max(X[:,0])
    _DMP_.append([g[0]*_GMPv[r]/max(X[:,0]), g[1]*nv, g[2]*nv, g[3]*nv])
#     _DMP_.append(g*(_v_[r]/max(X[:,0])))
_DMP_ = np.array(_DMP_)

In [12]:
_DMP_

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.44744117e+00, 7.46228864e-03, 2.98258099e-04, 5.32774647e-05],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.58677493e+00, 3.01982892e-01, 1.20698686e-02, 2.15602527e-03],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e

## Step 4: SP

In [13]:
# SP
CO2_SP = []
SO2_SP = []
NOX_SP = []
states = []
for r in R:
    if(_DMP_[r,0]*_DMP_[r,1] > 0):
        states.append(r)
        CO2_SP.append(tuple(['State '+str(r+1), P[r,0]*(_DMP_[r,0]/_DMP_[r,1])]))
        SO2_SP.append(tuple(['State '+str(r+1), P[r,0]*(_DMP_[r,0]/_DMP_[r,2])]))
        NOX_SP.append(tuple(['State '+str(r+1), P[r,0]*(_DMP_[r,0]/_DMP_[r,3])]))
    

In [14]:
CO2_SP

[('State 5', 25236.48742766957),
 ('State 11', 1319.7472918463866),
 ('State 15', 169.97654840216578),
 ('State 17', 268.43864157387753),
 ('State 21', 1096.400979505637),
 ('State 27', 55.19168067016959),
 ('State 33', 1844.6232868718735)]

In [15]:
SO2_SP

[('State 5', 631405.9996600309),
 ('State 11', 33019.50639902645),
 ('State 15', 4252.7397194333635),
 ('State 17', 6716.218701834867),
 ('State 21', 27431.47827038625),
 ('State 27', 1380.871977779978),
 ('State 33', 46151.67676490945)]

In [16]:
NOX_SP

[('State 5', 3534739.399408474),
 ('State 11', 184849.92268128955),
 ('State 15', 23807.703204917812),
 ('State 17', 37598.760343110705),
 ('State 21', 153567.00297203346),
 ('State 27', 7730.40260628833),
 ('State 33', 258366.48732753046)]

In [17]:
i = 0
ttl_C = 0
ttl_S = 0
ttl_N = 0
Sum = 0
for state in states:
    ttl_C += X[state][0]*CO2_SP[i][1]
    ttl_S += X[state][0]*SO2_SP[i][1]
    ttl_N += X[state][0]*NOX_SP[i][1]
    Sum += X[state][0]
    i += 1
CO2SP, SO2SP, NOXSP = ttl_C/Sum, ttl_S/Sum, ttl_N/Sum
CO2SP, SO2SP, NOXSP = CO2SP/1.10231131092, SO2SP/1.10231131092, NOXSP/1.10231131092

print('===== Result the shadow price in '+year+' =====')
print('CO2 = '+str(CO2SP))
print('SO2 = '+str(SO2SP))
print('NOX = '+str(NOXSP))

===== Result the shadow price in 2010 =====
CO2 = 349.0204562020728
SO2 = 8732.340849006176
NOX = 48885.4227939958
